In [3]:
from bs4 import BeautifulSoup as bs
import requests
import pprint as pp
import re

from pymongo import MongoClient
from pprint import pprint

client = MongoClient('localhost', 27017)
db = client['vacancy_db']

vacancies = db.vacancies

main_link = 'https://hh.ru/search/vacancy'

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
    'Accept': '*/*'}

vaks = []


def page_worker(href):
    response = requests.get(href, headers=header)
    soup = bs(response.text, 'html.parser')

    vak_block = soup.find('div', {'class': 'vacancy-serp'})
    vak_list = vak_block.find_all('div', {'class': 'vacancy-serp-item'})

    for vak in vak_list:
        vak_data = {}
        vak_name = vak.find('a', {'class': 'bloko-link HH-LinkModifier'})
        vak_sum = vak.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        if (vak_sum):
            vak_sum = vak_sum.getText()
        else:
            vak_sum = ''

# 3) Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта
        vak_data['_id'] = re.search(r'\d+', vak_name['href']).group()
        vak_data['vak_name'] = vak_name.getText()

        vak_sum_txt1 = re.sub(r"\s+", "", vak_sum.replace('от', '', 1).replace('до', '', 1).replace('руб.', '', 1))
        vak_sum_txt2 = re.split("-", vak_sum_txt1)

        if vak_sum.find("от") != -1:
            vak_data['vak_sum_min'] = vak_sum_txt2[0]
            vak_data['vak_sum_max'] = None
        elif vak_sum.find("до") != -1:
            vak_data['vak_sum_min'] = None
            vak_data['vak_sum_max'] = vak_sum_txt2[0]
        elif vak_sum.find("-") != -1:
            vak_data['vak_sum_min'] = vak_sum_txt2[0]
            vak_data['vak_sum_max'] = vak_sum_txt2[1]

        vak_data['vak_sum_val'] = vak_sum[-4:]

        vak_data['vak_ref'] = vak_name['href']
        vak_data['vak_source'] = 'hh.ru'
        vaks.append(vak_data)


#2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)
def vacancies_find(min_sum, max_sum):
    v = None

    if (min_sum > 0 and max_sum > 0):
        v = vacancies.find({'vak_sum_min': { '$gte': str(min_sum)}, 'vak_sum_max': { '$lte': str(max_sum)}})
    elif (min_sum > 0):
        v = vacancies.find({'vak_sum_min': { '$gte': str(min_sum)}})
    elif (max_sum > 0):
        v = vacancies.find({'vak_sum_max': { '$lte': str(max_sum)}})

    pp.pprint(v)

    if v is not None:
        for v1 in v:
            pp.pprint(v1)


vakancy_name = 'Data scientist'.replace(' ', '+')
vakancy_href = 'https://hh.ru/search/vacancy?clusters=true&enable_snippets=true&salary=&st=searchVacancy&text=' + vakancy_name + '&fromSearch=true&from=suggest_post&customDomain=1'

response = requests.get(vakancy_href, headers=header)
soup = bs(response.text, 'html.parser')

i = 1
vak_page = soup.find('a', {'class': 'bloko-button HH-Pager-Control', 'data-page': i})

while vak_page:
    page_worker('https://hh.ru/' + vak_page['href'])
    i += 1
    vak_page = soup.find('a', {'class': 'bloko-button HH-Pager-Control', 'data-page': i})

# 3) Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта
for vak in vaks:
    try:
        vacancies.insert_one(vak)
    except:
        continue
        #print("Duplicate key error index on _id")




vacancies_find(50000, 0)

#pp.pprint(vaks)



{'_id': '37327686',
 'vak_name': 'Математик-аналитик (data scientist / data analyst)',
 'vak_ref': 'https://nemchinovka.hh.ru/vacancy/37327686?query=Data%20scientist',
 'vak_source': 'hh.ru',
 'vak_sum_max': None,
 'vak_sum_min': '85000',
 'vak_sum_val': 'руб.'}
{'_id': '37707788',
 'vak_name': 'Диджитал-аналитик (Digital Analyst)',
 'vak_ref': 'https://nemchinovka.hh.ru/vacancy/37707788?query=Data%20scientist',
 'vak_source': 'hh.ru',
 'vak_sum_max': '150000',
 'vak_sum_min': '75000',
 'vak_sum_val': 'руб.'}
{'_id': '34338759',
 'vak_name': 'Руководитель аналитического направления',
 'vak_ref': 'https://nemchinovka.hh.ru/vacancy/34338759?query=Data%20scientist',
 'vak_source': 'hh.ru',
 'vak_sum_max': None,
 'vak_sum_min': '80000',
 'vak_sum_val': 'руб.'}
{'_id': '37506599',
 'vak_name': 'Директор по инвестициям (Family office)',
 'vak_ref': 'https://nemchinovka.hh.ru/vacancy/37506599?query=Data%20scientist',
 'vak_source': 'hh.ru',
 'vak_sum_max': '1000000',
 'vak_sum_min': '500000',